In [ ]:
!pip install deepface
!pip install ipywidgets
!pip install opencv-python
!pip install imutils

In [ ]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Image, Javascript
from deepface import DeepFace
from PIL import Image as PILImage
import io
from google.colab.output import eval_js
from base64 import b64decode

24-09-19 12:24:07 - Directory /root/.deepface has been created
24-09-19 12:24:07 - Directory /root/.deepface/weights has been created


In [ ]:
# Create header and subheader
project_name = widgets.HTML("<h1>Face Emotion Detection</h1>")
subheader = widgets.HTML("<h3>Choose an option to identify the emotion!</h3>")
display(project_name)
display(subheader)

HTML(value='<h1>Face Emotion Detection</h1>')

HTML(value='<h3>Choose an option to identify the emotion!</h3>')

In [ ]:
# Create buttons and file upload widget
button_upload = widgets.Button(description='Upload Image', button_style='primary')
button_webcam = widgets.Button(description='Open Webcam', button_style='success')
file_upload = widgets.FileUpload(accept='image/*', multiple=False)

In [ ]:
# Create output widget
output = widgets.Output()

# Display the buttons and file upload widget
display(button_upload, button_webcam, file_upload, output)

# Function to handle image upload
def on_upload_click(change):
  image_file = list(file_upload.value.values())[0]
  image_data = image_file['content']

  # Convert the image data to a PIL image
  img = PILImage.open(io.BytesIO(image_data))

  # Convert to OpenCV format
  img_cv = np.array(img)
  img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)

  # Perform emotion detection
  analyze_emotion(img_cv)

# Function to capture image from webcam
def on_webcam_click(b):
  filename = take_photo()

  # Load the captured image
  img_cv = cv2.imread(filename)

  # Perform emotion detection
  analyze_emotion(img_cv)

# Function to analyze emotion
def analyze_emotion(img):
  predictions = DeepFace.analyze(img, actions=['emotion'])

  # Display the dominant emotion
  dominant_emotion = predictions[0]['dominant_emotion']
  print(f"Dominant emotion: {dominant_emotion}")

  # Display image with emotion
  img_display = cv2.putText(img, dominant_emotion, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
  _, im_arr = cv2.imencode('.jpg', img_display) # Convert image to Numpy array
  im_bytes = im_arr.tobytes()
  display(Image(data=im_bytes))

# JavaScript to capture video stream from webcam
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to a fixed size.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
      f.write(binary)
      return filename

# Function for predicting from upload
def predict_from_upload():
  # Trigger file upload (or do any other logic)
  with output:
    print("Please upload an image using the file upload button.")

Button(button_style='primary', description='Upload Image', style=ButtonStyle())

Button(button_style='success', description='Open Webcam', style=ButtonStyle())

FileUpload(value={'angry123.png': {'metadata': {'name': 'angry123.png', 'type': 'image/png', 'size': 1312038, …

Output()

In [ ]:
# Event handlers for buttons
button_upload.on_click(lambda b: predict_from_upload())
button_webcam.on_click(on_webcam_click)

# Set up event listener for file upload
file_upload.observe(on_upload_click, names='value')